# Data cleaning and merging dataframes

## Loading multiple datasets

### Classical way

Reading file by file

In [1]:
import pandas as pd

path = 'raw_data/'
orderlines = pd.read_csv(path + 'orderlines.csv')
orders = pd.read_csv(path + 'orders.csv')
brands = pd.read_csv(path + 'brands.csv')
products = pd.read_csv(path + 'products.csv')

### Another way

Using `os` and a loop to read all the files from a directory. It's also possible to read only files with a certain extension (like `.csv`):

In [367]:
import pandas as pd
import os
path = 'raw_data/'
path, dirs, files = next(os.walk(path))
#print(files)

# remove non-csv files
for file in files:
    if not file.endswith("csv"):
        files.remove(file)

In [368]:
files

['brands.csv', 'orderlines.csv', 'orders.csv', 'products.csv']

In [369]:
# create empty list 
df_list = []

# append datasets to the list
for file in files:
    temp_df = pd.read_csv(path + file, sep=',')
    df_list.append(temp_df)

# show results
brands, orderlines, orders, products = df_list[0], df_list[1], df_list[2], df_list[3]

## Data quality

### Missing values

In [370]:
def check_missing_values(data): 
    print('Missing values:' + '\n' + str(data.isna().sum()))

In [371]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    print('\n')

File name: brands.csv
Missing values:
short    0
long     0
dtype: int64


File name: orderlines.csv
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64


File name: orders.csv
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


File name: products.csv
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64




### Duplicates

In [372]:
def check_duplicates(data): 
    print('Duplicated rows: ', data.duplicated(keep=False).sum())

In [373]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    print('\n')

File name: brands.csv
Missing values:
short    0
long     0
dtype: int64
Duplicated rows:  0


File name: orderlines.csv
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0


File name: orders.csv
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64
Duplicated rows:  0


File name: products.csv
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64
Duplicated rows:  9503




Based on our initial exploration we need to find a way to deal with our missing values. 

Do we have other problems like data inconsistency? 

* Example: problems with the `price` column in the `orderlines` dataframe.

### Data inconsistency - wrong data types

In [374]:
for i in range(len(df_list)): 
    
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    df_list[i].info()
    print('\n')

File name: brands.csv
Missing values:
short    0
long     0
dtype: int64
Duplicated rows:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


File name: orderlines.csv
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int

Some aspects that do not make a lot of sense:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be floats. 
    * Quite a lot of missing values: 
        * `description`: 7 missing values. Maybe that could be inferred from the product name?
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 

    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` matches with the created dates for `orderlines`. 
    
    
* **orderlines**: 
    * `unit_price` has to be a float, something wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it matches with the `orders` dataset. 
    
    
* **brands**: looks fine. 


Where do we have to start? Since `orders` and `orderlines` seem very crucial to the analysis, we will start by checking that they match.

## Cleaning orders

In [375]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

Some data I could extract from here to do a data quality check with `orderlines`: 

* the sum of `total_paid` on both datasets should be the same
* the initial and last date should be the same

In [376]:
min(orders['created_date'])

Timestamp('2017-01-01 00:07:19')

In [377]:
max(orders['created_date'])

Timestamp('2018-03-14 13:58:36')

In [378]:
sum(orders['total_paid'])

nan

I forgot to deal with missing values on our orders dataset!

In [379]:
orders.shape

(226909, 4)

In [380]:
orders.isna().sum()

order_id        0
created_date    0
total_paid      5
state           0
dtype: int64

In [381]:
# let's filter them out
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


These orders have a state of "Pending", so I can exclude them from our dataset. We are interested in the completed ones.

In [382]:
orders.dropna(inplace=True)

Now my dataset is clean. And the total paid is: 

In [383]:
orders.shape

(226904, 4)

In [125]:
orders['total_paid'].sum()

129159615.07000001

## Cleaning orderlines

First let's transform our date time. 

In [126]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [127]:
min(orderlines['date'])
# orders: Timestamp('2017-01-01 00:07:19')

Timestamp('2017-01-01 00:07:19')

In [128]:
max(orderlines['date'])
# orders: Timestamp('2018-03-14 13:58:36')

Timestamp('2018-03-14 13:58:36')

Yes, it seems it matches. Now let's test our total_paid. 

To get this value, we will have to calculate a new column, total price for each row. It would be `product_quantity` * `unit_price`. I can not multiply two different types of data. So I will have to transform the unit price to a numeric one. 

In [129]:
orderlines['product_quantity'] * pd.to_numeric(orderlines['unit_price'])

ValueError: Unable to parse string "1.137.99" at position 6

While trying to transform this column to numeric an error appears. From the error message, "Unable to parse string "1.137.99" at position 6", it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and python only admits one dot per number: the _decimal_ separator!

There are a lot of different ways to solve this problem. The first thing I will do is to count how many dots appear for each number using string operations. If price has more than one dot, I will consider it corrupted. 

In [130]:
# we create a copy of the dataset
ol_temp = orderlines.copy()

# create a new column with the amount of dots in the unit_price column
ol_temp['dots'] = orderlines['unit_price'].str.count('\.')

# show the rows with more than one dot
ol_temp.query('dots > 1')

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21,2
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42,2
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47,2
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00,2
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11,2
...,...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33,2
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15,2
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56,2
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51,2


Our theory about the thousands separators is confirmed. How can we solve this problem? 

My approach would be to remove all the dots for all the unit_prices, and then add a dot before the last 2 digits to all the rows. Then I will transform it into numeric values.

In [131]:
orderlines = orderlines.assign(unit_price_nd = orderlines['unit_price'].str.replace('\.','', regex=True))
orderlines.sample(5)

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd
243471,1569440,494106,0,1,OWC0257,18.99,2018-01-18 19:08:18,1899
8286,1140620,307116,0,9,SAN0112,19.99,2017-01-11 10:32:21,1999
23221,1171424,319983,0,1,QNA0150,314.99,2017-02-04 20:50:07,31499
24851,1174490,319721,0,1,OWC0056-2,19.99,2017-02-08 17:16:18,1999
204364,1500207,464622,0,1,POL0014,139.99,2017-12-17 00:18:59,13999


In [132]:
orderlines['integers'] = orderlines['unit_price_nd'].str[:-2]
orderlines['digits'] = orderlines['unit_price_nd'].str[-2:]
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,integers,digits
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,18,99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,399,00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,474,05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,68,39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,23,74


In [133]:
orderlines['new_unit_price'] = orderlines['integers'] + '.' + orderlines['digits']
orderlines.sample(5)

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,integers,digits,new_unit_price
236121,1557401,489583,0,1,APP0662,138.00,2018-01-12 13:56:29,13800,138,00,138.00
137709,1390210,418469,0,1,SPH0013,119.00,2017-10-27 09:37:23,11900,119,00,119.00
111077,1330120,395215,0,1,LAC0224,338.40,2017-08-31 21:30:29,33840,338,40,338.40
81391,1276193,369617,0,1,SAM0086,704.99,2017-06-27 17:35:21,70499,704,99,704.99
213698,1518033,472951,0,1,APP2072,965.59,2017-12-27 09:33:54,96559,965,59,965.59


In [134]:
# last step is to transform it to numeric and overwrite the original column unit_price with the correct prices
orderlines['unit_price'] = pd.to_numeric(orderlines['new_unit_price'])
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
 7   unit_price_nd     293983 non-null  object        
 8   integers          293983 non-null  object        
 9   digits            293983 non-null  object        
 10  new_unit_price    293983 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 24.7+ MB


Now it is time to see if it is the same than the orders files. 

In [135]:
#drop columns that we don't need anymore
orderlines.drop(['unit_price_nd','digits','integers','new_unit_price'], axis=1, inplace=True)

orderlines['total_price'] = orderlines['product_quantity'] * orderlines['unit_price']
sum(orderlines['total_price'])
# 129159615.07000001

128776222.0298095

In [136]:
sum(orders['total_paid'])

129159615.06991275

What!? After all this work the values do not match! How can we figure it out where is the problem? 

## Matching `orders` and `orderlines`

It is possible that some orders exist in one dataset but not in the other one. This would be a potential source for this price mismatch. Let's solve it!

We first create a new column in the `orderlines` dataset using `assign`. We also use `isin()` to create a boolen value (True/False) that checks whether the `id_order` is present in the `orders` dataset:

In [137]:
(orderlines.
 assign(check_orders = orderlines.id_order.isin(orders.order_id))
)['check_orders'].sum()

293743

In [138]:
orderlines.shape

(293983, 8)

There is a difference in the number of orders in orderlines and orders. Then, using `.query` we select rows where the value in this new column is `False`:

In [139]:
(orderlines.
 assign(check_orders = orderlines.id_order.isin(orders.order_id)).
 query("check_orders==False")
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27,2317.90,False
63,1119218,296284,0,1,BNQ0042,699.00,2017-01-01 09:58:35,699.00,False
67,1119223,294806,0,1,APP1849,2558.99,2017-01-01 10:09:15,2558.99,False
69,1119226,294806,0,1,APP1864,2797.99,2017-01-01 10:15:14,2797.99,False
70,1119235,297261,0,1,QNA0177,304.99,2017-01-01 10:17:59,304.99,False
...,...,...,...,...,...,...,...,...,...
275665,1621177,244328,0,1,OWC0260,349.00,2018-02-20 13:00:45,349.00,False
280055,1628699,261391,0,1,APP2352,3343.00,2018-02-27 11:43:25,3343.00,False
280352,1629247,287797,0,1,SYN0182,484.11,2018-02-27 19:16:26,484.11,False
280856,1630150,261391,0,1,SAT0091,44.99,2018-02-28 12:18:26,44.99,False


It looks like 240 rows in `orderlines` come from orders not present in the `orders` dataset. This is quite inconsistent, since the `orders` dataset should be the one and only source of truth for orders: if an order is not there, it should not exist. We will try to find out why that happened, but for now, let's just remove those "ghost" orders:

In [140]:
#only put orders in orderlines that are also in orders (check_orders == True), drop others
orderlines = (orderlines
              .assign(check_orders = orderlines.id_order.isin(orders.order_id))
              .query("check_orders==True")
             )

Now let's look at this problem in the opposite direction: are there orders in the `orders` dataset not present in `orderlines`?

In [141]:
(orders
 .assign(check_orders = orders.order_id.isin(orderlines.id_order))
 .query("check_orders==False")
)

,order_id,created_date,total_paid,state,check_orders
8,245941,2017-01-01 10:32:23,183.52,Completed,False
65,268629,2017-01-31 11:27:25,73.23,Completed,False
75,272862,2017-04-18 18:17:10,128.99,Pending,False
82,277655,2017-02-01 08:44:55,9.99,Pending,False
83,277994,2017-01-23 18:30:11,52.99,Completed,False
...,...,...,...,...,...
226835,527328,2018-03-14 13:14:12,0.00,Shopping Basket,False
226851,527344,2018-03-14 13:25:21,0.00,Place Order,False
226853,527346,2018-03-14 13:26:14,0.00,Place Order,False
226855,527348,2018-03-14 13:28:18,0.00,Place Order,False


There are more than 22000 orders in the `orders` dataset that are not present on the `orderlines` dataset!!! We can try to find out why by looking at the state of these orders:

In [142]:
(orders
 .assign(check_orders = orders.order_id.isin(orderlines.id_order))
 .query("check_orders==False")
 .state.value_counts()
)

Place Order        12304
Shopping Basket     9810
Completed             45
Cancelled             41
Pending               13
Name: state, dtype: int64

It looks like most of them are orders that were not fully completed: the products were left in the shopping basket or the order was "placed" but maybe not paid (hence the state "Place Order". Some of them were "Completed", though. 

This will require further research, and we might have to come back to these orders if we have to explore consumer behaviour (e.g. why are orders left in the shopping basket?), but for now, for the sake of data quality, let's drop all of these unmatched orders:

In [143]:
#only keep matching rows in orders
orders = (orders
          .assign(check_orders = orders.order_id.isin(orderlines.id_order))
          .query("check_orders==True")
         )

Let's now check again if the total paid matches:

In [144]:
orders.total_paid.sum()

129130877.18

In [145]:
orderlines.total_price.sum()

128659393.77

STILL NOT MATCHING!!! This is outrageous. Let's keep exploring.

## Solving the price mismatch

Let's merge both datasets and compare, order by order, the `total_price`. We will call this new merged dataset `orders_info`.

*Note: To merge `orderlines` with `orders`, we have to group and aggregate `orderlines`!

In [146]:
orders_info = (
orderlines
    .groupby('id_order')
    .agg({'total_price':'sum'})
    .merge(orders, how='inner', left_on='id_order', right_on='order_id')
    .copy()
)
orders_info

,total_price,order_id,created_date,total_paid,state,check_orders
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True
...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True


Now that the `total` from both datasets is in the same dataframe, we can create a new column with the difference:

In [147]:
orders_info= (
orders_info
    .assign(price_difference = round(orders_info['total_paid'] - orders_info['total_price'], 2)) 
)

In [244]:
#look only hat completed orders
(
    orders_info
        .query('state == "Completed"')
        .sort_values(by='price_difference').tail(50)
)

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
201280,147.04,523652,2018-03-09 10:40:47,167.05,Completed,True,20.01
191358,15.97,512413,2018-02-16 09:24:56,36.95,Completed,True,20.98
191719,7.98,512871,2018-02-16 12:57:43,28.96,Completed,True,20.98
191546,16.98,512656,2018-02-16 10:04:08,37.97,Completed,True,20.99
192511,210.95,513754,2018-02-18 14:50:47,231.94,Completed,True,20.99
188103,15.96,508741,2018-02-16 11:31:26,36.95,Completed,True,20.99
146,41.99,297200,2017-01-19 02:21:44,64.97,Completed,True,22.98
192487,18.98,513728,2018-02-18 14:06:44,42.97,Completed,True,23.99
101,529.99,286098,2017-01-16 14:34:55,553.98,Completed,True,23.99
191391,19.96,512456,2018-02-16 08:56:03,44.95,Completed,True,24.99


Price difference could come from shipping costs, prices paid are higher than total prices that were calculated from unit_price and product_quantity.

In [149]:
orders_info.price_difference.describe()

count    204691.000000
mean          2.303391
std         297.461827
min      -36159.900000
25%           0.000000
50%           0.000000
75%           0.010000
max       90898.610000
Name: price_difference, dtype: float64

Looks like the maximum and minimun price differences are huge: some orders are really corrupted. But we also see from the mean and the quartiles that the price difference is neglegible for most orders.

## Challenge: Remove outliers

Decide on a criteria for removing orders whenever you cannot trust the price difference between `orders` and `orderlines`. 

In [391]:
orders_info.query('price_difference >= 0 & price_difference < 105 & state == "Completed"').count() / orders_info.query('state == "Completed"').price_difference.count()

total_price         0.987264
order_id            0.987264
created_date        0.987264
total_paid          0.987264
state               0.987264
check_orders        0.987264
price_difference    0.987264
dtype: float64

Shipping costs for example with gls in Spain for individuals could be something up to 100€ when shipped to the Balearics. So price differences between 0 and 100 would be ok, others would be droped. Also taking into account that the really high outliers are in state 'Sopping Basket'. And price differences between 0 and 100 already make out 98,7%

Save the data once you are sure you can trust it!

In [ ]:
orderlines.to_csv(path + 'clean/orderlines_cl.csv', index=False)
orders.to_csv(path + 'clean/orders_cl.csv', index=False)

## Challenge: Cleaning products

Now it is time to clean the products dataset. Let's do a quick review of the things where wrong: 

In [165]:
print(products.info())
check_duplicates(products)
check_missing_values(products)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
None
Duplicated rows:  9503
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64


Looking at this overview, we can see that there are different things that has to be changed: 

* Data types: 
    * price should be a float
    * promo price should be a float
* Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 
* Missing values: 
    * Description may be can be infered by the name
    * price. Is there a way we can extract the information from another table?
    * type. Do we need this column for our analysis?
    
This task can be accomoplished by using all the methods you already know.

### Start of the challenge

In [200]:
# code here
#filter rows with missing values in column desc
sku_list = products.loc[products['desc'].isna()].sku.to_list()
sku_list

['WDT0211-A',
 'APP1622-A',
 'PAC2334',
 'KAN0034-A',
 'HTE0025',
 'OTT0200',
 'HOW0001-A']

In [389]:
for s in sku_list:
    products['desc'] = products['name']

products['desc'].isna().sum()    

0

In [390]:
products.iloc[16126]

sku                                                    WDT0211-A
name           Open - Purple 2TB WD 35 "PC Security Mac hard ...
desc           Open - Purple 2TB WD 35 "PC Security Mac hard ...
price                                                        107
promo_price                                              814.659
in_stock                                                       0
type                                                        1298
Name: 16126, dtype: object

In [237]:
products.loc[products['desc'].isna()]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...",NaN,107,814.659,0,1298
16128,APP1622-A,Open - Apple Smart Keyboard Pro Keyboard Folio...,NaN,1.568.206,1.568.206,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.35,5.659.896,0,12175397
18152,KAN0034-A,Open - Kanex USB-C Gigabit Ethernet Adapter Ma...,NaN,29.99,237.925,0,1298
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror an...,NaN,24.99,22.99,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.99,56.99,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador ...,NaN,199.99,1.441.174,0,11905404


In [387]:
prod_orderlines = products.merge(orderlines, how='inner', on = 'sku') 
prod_order_orderlines = prod_orderlines.merge(orders, how = 'left', left_on='id_order', right_on='order_id')
prod_order_orderlines.head()

,sku,name,desc,price,promo_price,in_stock,type,id,id_order,product_id,product_quantity,unit_price,date,order_id,created_date,total_paid,state
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,1121326,300551,0,1,54.99,2017-01-02 13:34:30,300551.0,2017-01-02 13:37:16,54.99,Completed
1,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,1133226,304067,0,1,49.99,2017-01-07 09:02:08,304067.0,2017-01-07 09:02:08,49.99,Shopping Basket
2,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,1134280,304484,0,1,49.99,2017-01-07 21:17:55,304484.0,2017-01-07 21:17:55,49.99,Shopping Basket
3,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,1136418,305406,0,1,49.99,2017-01-09 07:45:12,305406.0,2017-01-09 07:43:42,79.98,Shopping Basket
4,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,1136825,305590,0,1,49.99,2017-01-09 11:53:15,305590.0,2017-01-09 11:49:31,629.98,Shopping Basket


In [388]:
prod_order_orderlines.query('state == "Completed"').loc[prod_order_orderlines['desc'].isna()]

,sku,name,desc,price,promo_price,in_stock,type,id,id_order,product_id,product_quantity,unit_price,date,order_id,created_date,total_paid,state
282554,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...",NaN,107,814.659,0,1298,1318930,389901,0,1,81.47,2017-08-16 13:23:17,389901.0,2017-08-16 13:27:15,81.47,Completed
282556,APP1622-A,Open - Apple Smart Keyboard Pro Keyboard Folio...,NaN,1.568.206,1.568.206,0,1298,1255529,359388,0,1,156.82,2017-05-26 12:55:06,359388.0,2017-05-26 12:55:55,156.82,Completed
366699,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror an...,NaN,24.99,22.99,1,1515,1543840,484351,0,1,20.99,2018-01-07 16:40:59,484351.0,2018-01-07 16:55:20,49.96,Completed
370299,HOW0001-A,Open - Honeywell thermostat Lyric zonificador ...,NaN,199.99,1.441.174,0,11905404,1577535,497327,0,1,144.12,2018-01-23 23:11:50,497327.0,2018-01-23 23:41:17,151.11,Completed


In [183]:
products.loc[products['name'].str.contains('Open')]

,sku,name,desc,price,promo_price,in_stock,type
630,BEL0127-A,Open - Belkin MIXIT Lightning iPhone Support,Loading dock + support synchronization with Li...,34.99,299.899,0,1298
1309,SEA0043-A,"Open - Seagate Barracuda 1TB 35 ""SATA 7200rpm ...",internal hard drive for Mac and PC Refurbished...,59,442.836,0,1298
1823,WDT0175-A,"Open - Western Digital 2TB Green 35 ""5400rpm h...",WD Internal Hard Drive 2TB Mac and PC.,90,777.933,0,1298
1824,WDT0181-A,"Open - Western Digital Blue 1TB 25 ""SATA III 5...",Reconditioned WD 1TB internal hard drive for M...,80,545.846,1,12655397
1878,NTE0056-A,Open - NewerTech NuPower Battery 65W MacBook P...,MacBook Pro 13 inch Battery 2009/14,131.99,809.662,0,1298
...,...,...,...,...,...,...,...
19254,CRU0054-A,Open - Crucial MX500 250GB SSD 7mm,Hard disk SATA 250GB SSD refitted III (6 Gb / ...,86.53,66.99,0,12215397
19256,SAN0106-A,Open - SanDisk Ultra Flair Flash Drive 64GB US...,Reconditioned pendrive USB 3.0 64GB USB Flash ...,24.99,207.923,0,57445397
19257,NKI0012-A,Open - Nokia 36mm HR Steel Smartwatch Smart Cl...,Refurbished Nokia smart watch 36 mm Steel HR w...,189.95,1.269.864,0,11905404
19258,TWS0106-A,Open - Twelve South Bridge Magic Keyboard for ...,Light reconditioned support to hold the Keyboa...,48,260.978,0,8696


In [204]:
#filter rows with missing values in column price
products.loc[products['price'].isna()]

,sku,name,desc,price,promo_price,in_stock,type
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,NaN,299.899,0,8696
1900,AII0008,Aiino Case MacBook Air 11 '' Transparent,MacBook Air 11-inch casing with matte finish.,NaN,22.99,0,13835403
2039,CEL0020,Celly Ambo Luxury Leather Case + iPhone 6 Case...,Cover and housing together with magnet for iPh...,NaN,399.905,0,11865403
2042,CEL0007,Celly Wallet Case with removable cover Black i...,Case Book for iPhone 6 card case type.,NaN,128.998,0,11865403
2043,CEL0012,Celly Silicone Hard Shell iPhone 6 Blue,Hard Shell Silicone iPhone 6.,NaN,4.99,0,11865403
2044,CEL0014,Celly Silicone Hard Shell iPhone 6 Amarillo,Hard Shell Silicone iPhone 6.,NaN,59.895,0,11865403
2049,CEL0015,Celly fur-lined Powerbank battery 4000mAh Black,Leather-wrapped External Battery 4000mAh for i...,NaN,239.895,0,1515
2051,CEL0018,Celly Wallet Leather Case cover Black iPhone 6,Card case with transparent protective cover fo...,NaN,294.877,0,11865403
2052,CEL0023,Celly Ambo Luxury Leather Case + Case Gold iPh...,Cover and housing together with magnet for iPh...,NaN,329.894,0,11865403
2053,CEL0025,Celly Ambo Luxury Leather Case + Case iPhone 6...,Cover and housing together with magnet for iPh...,NaN,449.878,0,11865403


In [205]:
products.loc[products['name'].str.contains('Celly')].count()

sku            43
name           43
desc           43
price           0
promo_price    43
in_stock       43
type           43
dtype: int64

Items with missing prices are except 3 others only from company Celly (italien company selling phone cases and covers), sku starting with CEL, and Celly does not have any other products with prices so just drop the rows with missing price values

In [323]:
products_droped = products.dropna().copy()
products_droped.isna().sum()

sku            0
name           0
desc           0
price          0
promo_price    0
in_stock       0
type           0
dtype: int64

There are also a lot of duplicated rows (9503). Take a look at the duplicates.

In [324]:
products_droped[products_droped.duplicated()]

,sku,name,desc,price,promo_price,in_stock,type
101,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
102,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
103,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
104,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
105,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
...,...,...,...,...,...,...,...
16831,APP2302,"Apple MacBook Pro 13 ""Core i5 Touch Bar 33GHz ...",New MacBook Pro 13-inch Core i5 Touch Bar 33 G...,26.155.941,26.155.941,0,"1,02E+12"
16833,APP2303,"Apple MacBook Pro 13 ""Core i5 Touch Bar 33GHz ...",New MacBook Pro 13 inch Touch Bar 33 GHz Core ...,237.559.421,23.755.942,0,"1,02E+12"
18190,PAR0077,Parrot Bebop Drone 2 Power,Drone cuadricóptero quality camera integrated ...,699.9,6.733.892,0,11905404
18308,NKI0010,Nokia Wireless sphygmomanometer Plata,Sphygmomanometer for iPhone iPad and iPod App.,129.99,1.149.899,1,11905404


In [325]:
products_droped.count()

sku            19223
name           19223
desc           19223
price          19223
promo_price    19223
in_stock       19223
type           19223
dtype: int64

In [326]:
products_droped.drop_duplicates(inplace=True)

In [327]:
products_droped.count()

sku            10477
name           10477
desc           10477
price          10477
promo_price    10477
in_stock       10477
type           10477
dtype: int64

In [328]:
#investigate price column further
products_droped.sample(20)

,sku,name,desc,price,promo_price,in_stock,type
10953,OLL0011-A,(Open) Olloclip 4-in-1 lens iPhone 6 / 6S and ...,gLente wide angle lens macro fisheye and macro...,79.99,529.589,0,1298
3004,LOG0181,Ultimate Ears Wireless Speaker Roll Reef Verde,Logitech Bluetooth speaker ultra-compact porta...,129,899.877,0,5398
2439,HIC0019,Hitcase PRO + waterproof case with accessories...,Case resistant to extreme conditions and wide ...,149.99,1.299.903,0,11865403
12860,PAC1858,Synology DS216 + II | 12TB (2x6TB) Seagate Iro...,NAS with 12TB capacity (2x6TB) IronWolf Seagat...,8.929.696,7.361.785,0,12175397
1788,MUV0158,Muvit Pack 3 Adapter + SIM Card Reader,Adjustable NanoSim and MicroSim Adapters.,12.99,89.903,0,12585395
15300,DLK0138,D-Link DAP-1365 N300 Amplifier Repeater WiFi,Wi-Fi amplifier with Ethernet port supports wi...,61.99,289.892,0,1334
17307,SEA0098-A,Open - Seagate IronWolf 6TB NAS hard drive Sata 3,NAS hard drive designed for systems with inter...,279.99,1.722.724,0,1298
2216,APP0956,"Apple MacBook Air 13 ""i7 22 Ghz | 4GB RAM | 51...",laptop MacBook Air 13 inch i7 22GHz 4GB 512GB ...,1829,17.739.931,0,1282
16084,AP20194,Like new - Apple iPhone 6s 64GB Silver,Apple iPhone 6s Free Refurbished 64GB Color Si...,859,509.001,0,24821716
15536,SXA0014,SecurityXtra NoteSaver Twin Double safety Cabl...,Security cable to secure two Mac and PC devices,44.99,35.989,0,1325


In [329]:
orders_orderlines.sku.nunique()

7951

In [330]:
products_droped.sku.nunique()

10477

In [461]:
products_orders_lines = products.merge(orders_orderlines, how = 'inner', on = 'sku')
prices = products_orders_lines[['sku', 'name', 'price', 'promo_price','total_paid', 'product_quantity', 'unit_price','total_price']]
prices.sample(5)
#prices.query('sku == "XDO0047"')

,sku,name,price,promo_price,total_paid,product_quantity,unit_price,total_price
377399,MMW0016,"My MW Case MacBook Pro 13 ""(Late 2016) Black",29.99,20.99,7065.97,2,22.99,45.98
292550,APP2082,"Apple MacBook Pro 13 ""Core i5 with Touch Bar 3...",2005.59,18.850.046,1869.00,1,1869.00,1869.00
318022,APP2077,"Apple MacBook Pro 15 ""Core i7 Touch Bar 29Ghz ...",3305.59,3.074.005,3370.57,1,3018.59,3018.59
377426,MMW0016,"My MW Case MacBook Pro 13 ""(Late 2016) Black",29.99,20.99,2591.98,1,22.99,22.99
241436,SAT0008,Satechi USB Hub Slim-C to USB-A / HDMI 4K / US...,79.99,499.899,59.99,1,59.99,59.99


In [397]:
#first add decimals for all prices that don't have decimals (dots)
products_droped['price_new'] = products_droped.price.apply(lambda x: x if x.find('.') != -1 else x + '.00')
products_droped.sample(20)

,sku,name,desc,price,promo_price,in_stock,type,price_new,price_num,price_dec
18215,WDT0402,WD 4TB MyPassport Ultra Silver,4TB External Hard Drive USB 3.0 with Mac and P...,199.99,1.647.947,0,11935397,199.99,199,99
18337,FIT0063,High HR Fitbit activity fósica Bracelet Black ...,Bracelet High HR multisport activity Fitbit la...,149.95,1.249.906,1,11905404,149.95,149,95
2975,GTE0053,G-Technology G-RAID 4TB USB 3.0 RAID Extraóble,extraóble disk RAID FireWire 800 and USB 3.0 f...,408.99,3.199.942,0,12995397,408.99,408,99
13583,APP1716,Apple Watch Series 2 38mm Stainless Steel Case...,Apple Watch 38 mm with GPS dual-core processor...,1219,1219,0,24885185,1219.00,1219,00
12502,OTT0131,OtterBox Gel Skin Case + Protector Clearly Alp...,Screen protector anti-scratch + transparent ge...,34.99,22.99,1,13555403,34.99,34,99
16797,APP2254,"Apple MacBook Air 13 ""Core i7 22GHz | 8GB RAM ...",laptop MacBook Air 13 inch i7 22GHz 8GB RAM an...,1779,16.900.046,0,"2,17E+11",1779.00,1779,00
3082,APP1134,Case Apple iPhone 6 / 6s Brown Leather Case,ultrathin leather case and microfiber premium ...,55,509.894,0,11865403,55.00,55,00
12314,KIN0125,Kingston 8GB Micro SDHC Class 10 45MB / s,8GB micro SDHC card with adapter,4.99,4.281,0,57445397,4.99,4,99
3142,OTT0122,OtterBox Defender Case for iPhone 6 / 6S Plus ...,Funda iPhone 6 / 6S ultra resistant multilayer...,59.99,299.899,1,11865403,59.99,59,99
10132,PAC1592,"Apple iMac 27 ""Core i5 3.2GHz Retina 5K | 8GB ...",Desktop computer iMac 27-inch 3.2GHz Core i5 5...,3169,25.979.898,0,"5,74E+15",3169.00,3169,00


In [395]:
#second rsplit decimals (last 1, 2 or 3 numbers separated by a dot)
products_droped[['price_num', 'price_dec']]=products_droped.price_new.str.rsplit(pat='.', n=1, expand=True)

In [335]:
products_droped.sample(20)

,sku,name,desc,price,promo_price,in_stock,type,price_new,price_num,price_dec
15879,ALL0014,Allocacoc PowerCube Extended USB plug strip Wh...,Outlet power strip extension 15m cube-shaped w...,24.95,189.897,1,5395,24.95,24,95
1800,GRT0362,Griffin Survivor All-Terrain resistant Case iP...,resistant to ultra Air 2 cases. Case,79.99,599.918,0,12635403,79.99,79,99
15315,GLY0008,Glyph Atom 525GB SSD Hard Drive USB-C Gray,External Hard Drive 525GB SSD with USB-C conne...,304.99,2.399.914,0,11935397,304.99,304,99
2884,APP1104,Apple iPod Shuffle 2GB Gray Space,Music player iPod Shuffle 2GB and small size.,55,588.108,0,11821715,55.00,55,00
15685,HOC0008,Nike hoco Series Apple Watch Strap 42mm Black ...,Silicone Strap Sports Watch 42mm Apple,45,249.889,0,2449,45.00,45,00
1911,LGE0009-A,"Open - 34UM95 LG LCD Monitor 34 ""Ultra WQHD re...",Monitor ultrapanorámico IPS 21: 9 34-inch Thun...,919,6.729.081,0,1298,919.00,919,00
11662,JMO0092,Just Mobile iPhone Charger Stand Silver HoverDock,Smart charging stand for iPhone with Lightning...,39.95,328.999,0,13615399,39.95,39,95
11665,SAM0067-A,Open - Samsung 850 EVO SSD Disk 250GB,SSD hard drive Mac and PC 25 inch 250GB SATA I...,107.99,855.845,0,1298,107.99,107,99
1348,PUR0131,Puro Booklet Slim founded with support iPad Ai...,Smart Case with Stand for iPad Air 2.,44.95,299.959,0,12635403,44.95,44,95
17145,AP20222,Like new - Apple iPad mini 2 Wi-Fi 32GB Silver,Refurbished iPad Mini 2 32GB Wi-Fi Plata,289,2.328.113,0,12031714,289.00,289,00


In [500]:
products_prices = products_droped.merge(orderlines, on='sku', how='left')
products_prices.sample(5)

,sku,name,desc,price,promo_price,in_stock,type,price_new,price_num,price_dec,id,id_order,product_id,product_quantity,unit_price,date
99728,APP1216,Apple Magic Trackpad 2,Apple Wireless Bluetooth Trackpad.,149,139,1,1387,149.00,149,00,1589537.0,501842.0,0.0,1.0,130.00,2018-01-30 11:02:11
233427,APP2085,"Apple MacBook Pro 13 ""Core i5 with Touch Bar 3...",New MacBook Pro 13 inch Touch Bar 31 GHz Core ...,2255.59,20.980.045,0,"1,02E+12",2255.59,2255,59,1643801.0,524615.0,0.0,1.0,2.098.00,2018-03-10 08:42:26
99541,APP1216,Apple Magic Trackpad 2,Apple Wireless Bluetooth Trackpad.,149,139,1,1387,149.00,149,00,1587120.0,501003.0,0.0,1.0,130.00,2018-01-29 13:10:50
89040,APP1190,IPhone AppleCare Protection Plan,Apple Care extended warranty iPhone.,70,609.997,1,1231,70.00,70,00,1543574.0,484281.0,0.0,1.0,54.00,2018-01-07 14:23:35
175756,WDT0348,WD Blue 500GB SATA 3 SSD Disk 7mm,SSD 500GB SATA Hard Disk 3.0 (6Gb / s) for Mac...,169,149.994,0,12215397,169.00,169,00,1486555.0,458913.0,0.0,1.0,159.00,2017-12-10 12:38:24


In [490]:
three_decimals = products_droped.loc[products_droped['price_dec'].str.len()==3]
two_decimals = products_droped.loc[products_droped['price_dec'].str.len()==2]
one_decimal = products_droped.loc[products_droped['price_dec'].str.len()==1]
price_ok = products_droped.loc[products_droped['price_dec'].str.len()<=2]

In [498]:
price_ok.price.str.count('\.')

0        1
1        0
2        0
3        0
4        1
        ..
19321    1
19322    1
19323    1
19324    1
19325    1
Name: price, Length: 9940, dtype: int64

In [356]:
three_decimals.sku.count()

537

In [359]:
three_decimals.sample(20)

,sku,name,desc,price,promo_price,in_stock,type,price_new,price_num,price_dec
18939,SAM0114-A,"Open - Samsung LS22F352FHU Monitor 22 ""FHD HDM...",Monitor 22-inch Full HD VGA HDMI connection. 7...,109.989,1.069.895,0,1296,109.989,109,989
2683,QNA0110,TVS-663-4G QNAP NAS server Mac and PC,NAS server 6 bays for Mac and PC.,9.669.897,9.669.897,0,12175397,9.669.897,9.669,897
15329,REP0273,volume button repair on vibration control and ...,Repair service including parts and labor for i...,6.999.003,69.99,0,"1,44E+11",6.999.003,6.999,003
15902,PAC2006,Synology DS1517 + | 8GB RAM | 40TB (5x8TB) Sea...,NAS with 8GB RAM and 40TB for Mac and PC,27.187.885,20.069.629,0,12175397,27.187.885,27.187,885
15476,REP0335,Repair iPad 3 Headphone Input,Repair service including parts and labor for i...,6.999.003,69.99,0,"1,44E+11",6.999.003,6.999,003
13201,PAC1515,Pack QNAP TS-251A NAS Server | 8GB RAM | WD 16...,NAS with 8GB RAM and two hard disks 8TB (2x8TB...,11.956.689,9.679.903,0,12175397,11.956.689,11.956,689
2399,DEV0011,Devolo Wifi Repeater 24 GHz,Amplifier Wi-Fi N 24 GHz Devolo.,399.905,329.846,1,1334,399.905,399,905
12128,UAG0037,"Urban Armor Gear Case iPad Pro 97 ""Red / Black",Super durable Case for iPad Pro 97,499.899,499.899,0,12635403,499.899,499,899
12872,PAC2029,Synology DS216 + II | Seagate 4TB IronWolf for...,Synology DS216 + II with 2GB of RAM and 12TB (...,9.339.496,771.179,0,12175397,9.339.496,9.339,496
12983,TPL0016,TP-Link Gigabit Router Archer C5 Wi-Fi ac 1200...,Dual Band Wireless Gigabit Router AC1200 iPhon...,108.779,999.896,0,1334,108.779,108,779


In [ ]:
three_decimals

In [357]:
two_decimals.sku.count()

9434

In [493]:
price_ok.sku.count()

9940

In [491]:
new = price_ok.copy()
new['price_float'] = pd.to_numeric(new['price'])

In [492]:
new.sample(5)

,sku,name,desc,price,promo_price,in_stock,type,price_new,price_num,price_dec,price_float
17953,GOP0098,Floaty GoPro Hero & Hero 5 Session Session,Specific float Session GoPro Hero to enjoy it ...,29.99,249.889,0,5403,29.99,29,99,29.99
12344,SYN0141,Synology DS916 + | 2GB RAM Mac and PC Server Nas,4-bay NAS server and 2GB of RAM for Mac iPad a...,580.8,5.799.893,0,12175397,580.8,580,8,580.80
19124,JBL0134-A,Open - JBL T205 Black Headphones,JBL Reconditioned lightweight headphones so yo...,19.99,13.794,0,5384,19.99,19,99,19.99
8222,PAC1577,"Apple iMac 21.5 ""Core i7 3.3GHz 4K Retina | 8G...",Desktop computer iMac 21.5-inch 3.3GHz Core i7...,2449,19.565.944,0,1282,2449.00,2449,00,2449.00
18402,FIB0012,Fibaro HomeKit Wall Plug intelligent wall socket,Plug smart power meter real-time compatible wi...,69.99,699.864,1,11905404,69.99,69,99,69.99


In [464]:
one_decimal.sku.count()

506

## Brands

The brands csv looks fine, so we can work together with it. 

In [ ]:
brands.to_csv(path + 'clean/brands.csv', index=False)